# Spark DataFrames Project Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ProjectExercise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [0]:
df = spark.read.format("csv").option('header',True).option('inferSchema',True).load("dbfs:/FileStore/shared_uploads/yashbhatia96@g.ucla.edu/walmart_stock.csv")

#### What are the column names?

In [0]:
df.columns

Out[3]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [0]:
df.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

#### Print out the first 5 rows.

In [0]:
df.head(n=5)

Out[6]: [Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [0]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Date| Open| High| Low| Close| Volume| Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| null| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|2012-01-03|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max|2016-12-30| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [0]:
from pyspark.sql.functions import format_number
df_describe = df.describe()
df_describe.select(df_describe['summary'], format_number(df_describe['Open'].cast('float'),2).alias('Open'), format_number(df_describe['High'].cast('float'),2).alias('High'), format_number(df_describe['Low'].cast('float'),2).alias('Low'), format_number(df_describe['Close'].cast('float'),2).alias('Close'), format_number(df_describe['Volume'].cast('float'),2).alias('Volume'), format_number(df_describe['Adj Close'].cast('float'),2).alias('Adj Close')).show()

+-------+--------+--------+--------+--------+-------------+---------+
summary| Open| High| Low| Close| Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1,258.00| 1,258.00|
 mean| 72.36| 72.84| 71.92| 72.39| 8,222,093.50| 67.24|
 stddev| 6.77| 6.77| 6.74| 6.76| 4,519,781.00| 6.72|
 min| 56.39| 57.06| 56.30| 56.42| 2,094,900.00| 50.36|
 max| 90.80| 90.97| 89.25| 90.47|80,898,096.00| 84.91|
+-------+--------+--------+--------+--------+-------------+---------+

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
df_new = ((df['High'])/(df['Volume'])).alias('HV Ratio')
df_new = df.withColumn("HV Ratio", (df['High'])/(df['Volume']))
df_new.select("HV Ratio").show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

#### What day had the Peak High in Price?

In [0]:
df.orderBy(df['High'].desc()).head(1)[0][0]

Out[45]: '2015-01-13'

#### What is the mean of the Close column?

In [0]:
from pyspark.sql.functions import mean
df.select(mean(df['Close']).alias('Avg (Close)')).show()

+-----------------+
 Avg (Close)|
+-----------------+
72.38844998012726|
+-----------------+

#### What is the max and min of the Volume column?

In [0]:
from pyspark.sql.functions import min,max
df.select(max(df['Volume']), min(df['Volume'])).show()

+-----------+-----------+
max(Volume)|min(Volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

#### How many days was the Close lower than 60 dollars?

In [0]:
df.filter(df['Close'] < 60).count()

Out[62]: 81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [0]:
(df.filter('High > 80').count() / df.count())*100

Out[64]: 9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [0]:
from pyspark.sql.functions import corr
df.select(corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

#### What is the max High per year?

In [0]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year", year(df['Date']))
max_df = yeardf.groupby('Year').max()
max_df.select('Year', 'max(High)').show()

+----+---------+
Year|max(High)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [0]:
from pyspark.sql.functions import month, mean
monthdf = df.withColumn("Month", month(df['Date']))
meandf = monthdf.groupBy('Month').mean()
meandf.select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+

# Great Job!